# Chat with LLM-Recommender Hybrid
- Also see writeup here: https://eugeneyan.com/writing/semantic-ids/

In [1]:
CATEGORY = "Video_Games"

In [ ]:
import pip

def install_package_pip_api(package_name):
    """Installs a package using pip's internal API."""
    try:
        # pip.main expects a list of arguments, similar to how you use it on the command line
        # The arguments must be strings.
        pip.main(['install', package_name])
        print(f"Successfully installed {package_name} using the pip API.")
    except Exception as e:
        print(f"Failed to install {package_name}. Error: {e}")

install_package_pip_api("accelerate")
install_package_pip_api("transformers")
install_package_pip_api("datasets")
install_package_pip_api("pandas")
install_package_pip_api("torch")

import logging
import re
import sys
from typing import List

import pandas as pd
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

/Users/bytedance/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/bytedance/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Create logger with the same format as src/logger.py
logger = logging.getLogger("hf-chat")
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(logging.Formatter("%(asctime)s - %(message)s", datefmt="%H:%M:%S"))
logger.addHandler(handler)
logger.propagate = False  # Prevent duplicate messages

In [4]:
# Device selection
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
logger.info(f"Using device: {device}")

15:59:56 - Using device: mps


## Load Model and Dataset from Hugging Face Hub

In [5]:
# Define repository names based on category
category_lower = CATEGORY.lower().replace("_", "-")

# Repository names
MAPPING_REPO = f"eugeneyan/{category_lower}-semantic-ids-mapping"
MODEL_REPO = f"eugeneyan/semantic-id-qwen3-8b-{category_lower}"

logger.info("Loading from repositories:")
logger.info(f"  Dataset: {MAPPING_REPO}")
logger.info(f"  Model: {MODEL_REPO}")

15:59:56 - Loading from repositories:
15:59:56 -   Dataset: eugeneyan/video-games-semantic-ids-mapping
15:59:56 -   Model: eugeneyan/semantic-id-qwen3-8b-video-games


In [6]:
# Load the dataset from Hugging Face Hub
dataset = load_dataset(MAPPING_REPO, split="train")
mapping_df = dataset.to_pandas()

logger.info(f"✓ Loaded {len(mapping_df):,} products with semantic IDs and titles")
logger.info(f"Columns: {list(mapping_df.columns)}")
mapping_df.head()

15:59:58 - ✓ Loaded 66,097 products with semantic IDs and titles
15:59:58 - Columns: ['parent_asin', 'semantic_id', 'title']


,parent_asin,semantic_id,title
0,B000FH0MHO,<|sid_start|><|sid_32|><|sid_486|><|sid_636|><...,Dash 8-300 Professional Add-On
1,B00Z9TLVK0,<|sid_start|><|sid_188|><|sid_400|><|sid_518|>...,NBA 2K17 - Early Tip Off Edition - PlayStation 4
2,B002WH4ZJG,<|sid_start|><|sid_198|><|sid_262|><|sid_684|>...,Thrustmaster Elite Fitness Pack for Nintendo Wii
3,B0001ZNU56,<|sid_start|><|sid_65|><|sid_483|><|sid_757|><...,"Spongebob Squarepants, Vol. 1"
4,B009C9E8JY,<|sid_start|><|sid_154|><|sid_414|><|sid_585|>...,Set of 4 Bullet Buttons Nickel+Brass for Plays...


In [ ]:
# Load model and tokenizer from Hugging Face Hub
logger.info(f"Loading model from {MODEL_REPO}...")
logger.info("This may take a few minutes for the initial download (~16GB)")
        
model = AutoModelForCausalLM.from_pretrained(
    MODEL_REPO, torch_dtype=torch.bfloat16, device_map=device, trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_REPO,
    trust_remote_code=True,
    padding_side="left",  # Important for generation
)

# Ensure tokenizer has pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

logger.info("✓ Model loaded successfully")
logger.info(f"Model device: {next(model.parameters()).device}")

15:59:58 - Loading model from eugeneyan/semantic-id-qwen3-8b-video-games...
15:59:58 - This may take a few minutes for the initial download (~16GB)


`torch_dtype` is deprecated! Use `dtype` instead!
Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

## Helper Functions

In [ ]:
def parse_semantic_id(semantic_id: str) -> List[str]:
    """
    Parse a semantic ID string into its component levels.

    Example input: '<|sid_start|><|sid_127|><|sid_45|><|sid_89|><|sid_12|><|sid_end|>'
    Returns: ['<|sid_127|>', '<|sid_45|>', '<|sid_89|>', '<|sid_12|>']
    """
    # Remove start and end tokens
    sid = semantic_id.replace("<|sid_start|>", "").replace("<|sid_end|>", "")

    # Extract all sid tokens
    pattern = r"<\|sid_\d+\|>"
    levels = re.findall(pattern, sid)

    return levels


def map_semantic_id_to_titles(semantic_id_str: str, mapping_df: pd.DataFrame) -> dict:
    """
    Map a semantic ID to titles with 4-token exact match and 3-token fallback.

    Returns:
        dict with 'match_level', 'titles', and 'count' keys
    """
    # Parse the input semantic ID
    levels = parse_semantic_id(semantic_id_str)

    if not levels:
        return {"match_level": 0, "titles": [], "count": 0}

    # First try exact match (all 4 tokens)
    exact_matches = mapping_df[mapping_df["semantic_id"] == semantic_id_str]
    if len(exact_matches) > 0:
        titles = exact_matches["title"].tolist()
        return {"match_level": 4, "titles": titles, "count": len(titles), "match_type": "exact"}

    # Fallback to prefix matching (3 tokens, then 2, then 1)
    for depth in range(min(3, len(levels)), 0, -1):
        # Build the prefix for this depth
        prefix = "<|sid_start|>" + "".join(levels[:depth])

        # Find matches
        matches = mapping_df[mapping_df["semantic_id"].str.startswith(prefix)]

        if len(matches) > 0:
            # Found matches at this level
            titles = matches["title"].tolist()
            return {
                "match_level": depth,
                "titles": titles[:5],  # Limit to 5 for display
                "count": len(titles),
                "match_type": "prefix",
                "prefix_used": prefix,
            }

    # No matches found at any level
    return {"match_level": 0, "titles": [], "count": 0, "match_type": "none"}

In [ ]:
def extract_semantic_ids_from_text(text: str) -> List[str]:
    """
    Extract all semantic IDs from a text string.

    Returns list of full semantic IDs found in the text.
    """
    # Pattern to match complete semantic IDs
    pattern = r"<\|sid_start\|>(?:<\|sid_\d+\|>)+<\|sid_end\|>"
    semantic_ids = re.findall(pattern, text)
    return semantic_ids


def replace_semantic_ids_with_titles(text: str, mapping_df: pd.DataFrame = None, show_match_level: bool = True) -> str:
    """
    Replace all semantic IDs in text with their corresponding titles.

    Args:
        text: Input text containing semantic IDs
        mapping_df: DataFrame with semantic_id to title mapping
        show_match_level: Whether to append match info after the title

    Returns:
        Text with semantic IDs replaced by titles
    """
    if mapping_df is None:
        mapping_df = globals().get("mapping_df")
        if mapping_df is None:
            raise ValueError("No mapping_df provided or found in global scope")

    # Find all semantic IDs in the text
    semantic_ids = extract_semantic_ids_from_text(text)

    # Create a copy of the text to modify
    result_text = text

    # Replace each semantic ID with its title(s)
    for sid in semantic_ids:
        # Get matching titles
        match_result = map_semantic_id_to_titles(sid, mapping_df)

        if match_result["count"] > 0:
            # Use the first title if multiple matches
            title = match_result["titles"][0]

            # Add match level if requested
            if show_match_level:
                if match_result["match_type"] == "exact":
                    replacement = f'"{title}"'
                else:
                    replacement = f'"{title}" (L{match_result["match_level"]} match)'
            else:
                replacement = f'"{title}"'

            # If multiple matches, indicate this
            if match_result["count"] > 1:
                replacement += f" [+{match_result['count'] - 1} similar]"
        else:
            # No match found
            replacement = "[Unknown Item]"

        # Replace the semantic ID with the title
        result_text = result_text.replace(sid, replacement)

    return result_text

## How a Semantic ID looks like

In [ ]:
item = mapping_df.sample(1)
title = item['title'].item()
logger.info(f"Item Name: {title if len(title) < 80 else title[:80] + '...'}")
logger.info(f"Regular ID: {item['parent_asin'].item()}")
semantic_id_tokens = tokenizer.encode(item['semantic_id'].item())
logger.info(f"Semantic ID: {item['semantic_id'].item()}")
logger.info(f"Semantic ID as tokens: {semantic_id_tokens}")

09:08:39 - Item Name: LEGO Marvel's Avengers - Xbox 360 Digital Code
09:08:39 - Regular ID: B01AYMN0DM
09:08:39 - Semantic ID: <|sid_start|><|sid_220|><|sid_277|><|sid_739|><|sid_768|><|sid_end|>
09:08:39 - Semantic ID as tokens: [151670, 151892, 151949, 152411, 152440, 151671]


## Chat with Model

In [ ]:
GLOBAL_MESSAGES = []
SYSTEM_PROMPT = """
Qwen, you are a helpful AI assistant that understands and works with semantic IDs for product recommendations.

Semantic IDs are hierarchical identifiers in the format <|sid_start|><|sid_0|><|sid_256|><|sid_512|><|sid_768|><|sid_end|> that encode product information and relationships.
""".strip()

In [ ]:
def clean_output(text: str) -> str:
    """Remove known special tokens from output."""
    special_tokens = ["<think>", "</think>", "<|im_end|>", "<|im_start|>", "<|endoftext|>"]
    for token in special_tokens:
        text = text.replace(token, "")
    return text.strip()


def chat(
    text_input: str,
    new_convo: bool = True,
    temperature: float = 0.3,
    max_new_tokens: int = 512,
    stream: bool = True,
    mapping_df: pd.DataFrame = None,
) -> str:
    """
    Chat with the model, maintaining conversation history in GLOBAL_MESSAGES.

    Args:
        text_input: User input text
        new_convo: If True, clear global message history for a fresh start
        temperature: Generation temperature
        max_new_tokens: Maximum tokens to generate
        stream: Whether to stream output
        mapping_df: DataFrame for semantic ID mapping

    Returns:
        Generated response text
    """
    global GLOBAL_MESSAGES

    if mapping_df is None:
        mapping_df = globals().get("mapping_df")

    # Handle conversation history
    if new_convo:
        GLOBAL_MESSAGES = []
    else:
        # Display previous conversation turns if continuing
        if GLOBAL_MESSAGES:
            logger.info(f"{'=' * 41} Conversation History {'=' * 41}")
            for i, msg in enumerate(GLOBAL_MESSAGES, 1):
                role = msg["role"].upper()
                content = replace_semantic_ids_with_titles(msg["content"], mapping_df)
                logger.info(f"[Turn {(i+1)//2}] {role}: {content}")
            logger.info(f"{'=' * 35} Current Turn {'=' * 35}")

    GLOBAL_MESSAGES.append({"role": "user", "content": text_input})

    # Build messages with system prompt
    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + GLOBAL_MESSAGES

    # Log input with readable semantic IDs
    readable_input = replace_semantic_ids_with_titles(text_input, mapping_df)
    logger.info(f"USER: {readable_input}")
    logger.info(f"{'=' * 20} START RAW MODEL OUTPUT (WITH SEMANTIC IDS) {'=' * 20}")

    # Prepare input for model
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    # Generate response
    streamer = TextStreamer(tokenizer, skip_prompt=True) if stream else None

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=0.7,
            top_k=20,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            streamer=streamer,
        )

    # Decode only the newly generated tokens
    input_length = inputs["input_ids"].shape[1]
    generated_tokens = output[:, input_length:]
    generated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

    # Clean output
    generated_text = clean_output(generated_text)

    # Add to conversation history
    GLOBAL_MESSAGES.append({"role": "assistant", "content": generated_text})

    # Log output with readable semantic IDs
    logger.info(f"{'=' * 21} END RAW MODEL OUTPUT (WITH SEMANTIC IDS) {'=' * 21}")
    readable_output = replace_semantic_ids_with_titles(generated_text, mapping_df)
    logger.info(f"ASSISTANT: {readable_output}")

    return generated_text

## Single-Turn Conversations

In [ ]:
# Provide input of user past interactions and get recommendation
INPUT = """User: <|sid_start|><|sid_64|><|sid_313|><|sid_637|><|sid_768|><|sid_end|>, <|sid_start|><|sid_64|><|sid_447|><|sid_706|><|sid_768|><|sid_end|>, <|sid_start|><|sid_64|><|sid_355|><|sid_734|><|sid_768|><|sid_end|>\n<|rec|>""".strip()
response = chat(INPUT)

09:10:23 - USER: User: "Halo 3 Limited Edition -Xbox 360", "Lost Planet: Extreme Condition - Xbox 360", "Lost Planet: Extreme Condition Collector's Edition (Amazon.com Exclusive Cell Included) -Xbox 360"
<|rec|>
09:10:23 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<|sid_start|><|sid_64|><|sid_351|><|sid_733|><|sid_768|><|sid_end|><|im_end|>
09:10:24 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
09:10:24 - ASSISTANT: "Mass Effect - Xbox 360"


In [ ]:
# Provide input of single past item and get similar item
INPUT = """Customers who bought <|sid_start|><|sid_201|><|sid_311|><|sid_758|><|sid_768|><|sid_end|> also bought:\n<|rec|>""".strip()
response = chat(INPUT)

09:10:40 - USER: Customers who bought "New Super Mario Bros. 2" also bought:
<|rec|>
09:10:40 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<|sid_start|><|sid_201|><|sid_396|><|sid_608|><|sid_769|><|sid_end|><|im_end|>
09:10:41 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
09:10:41 - ASSISTANT: "The Legend of Zelda: Ocarina of Time 3D"


In [ ]:
# Steering recommendations given an item and attribute (Xbox)
INPUT = """Recommend Xbox games similar to <|sid_start|><|sid_201|><|sid_396|><|sid_608|><|sid_769|><|sid_end|>:\n<|rec|>""".strip()
response = chat(INPUT)

09:10:58 - USER: Recommend Xbox games similar to "The Legend of Zelda: Ocarina of Time 3D":
<|rec|>
09:10:58 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

<|sid_start|><|sid_64|><|sid_271|><|sid_576|><|sid_768|><|sid_end|>, <|sid_start|><|sid_64|><|sid_271|><|sid_576|><|sid_768|><|sid_end|>, <|sid_start|><|sid_64|><|sid_435|><|sid_615|><|sid_768|><|sid_end|><|im_end|>
09:11:00 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
09:11:00 - ASSISTANT: "Fallout: New Vegas - Xbox 360 Ultimate Edition", "Fallout: New Vegas - Xbox 360 Ultimate Edition", "Halo 4 - Xbox 360 (Standard Game)"


In [ ]:
# Steering recommendations given an item and attribute (Playstation)
INPUT = """Recommend PS4 games similar to <|sid_start|><|sid_201|><|sid_396|><|sid_608|><|sid_769|><|sid_end|>:\n<|rec|>""".strip()
response = chat(INPUT)

09:11:21 - USER: Recommend PS4 games similar to "The Legend of Zelda: Ocarina of Time 3D":
<|rec|>
09:11:21 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

<|sid_start|><|sid_139|><|sid_356|><|sid_594|><|sid_768|><|sid_end|>, <|sid_start|><|sid_139|><|sid_425|><|sid_652|><|sid_768|><|sid_end|>, <|sid_start|><|sid_139|><|sid_499|><|sid_524|><|sid_768|><|sid_end|><|im_end|>
09:11:23 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
09:11:23 - ASSISTANT: "Final Fantasy XV Deluxe Edition - PlayStation 4", "Bloodborne Collectors Edition - PlayStation 4", "UNCHARTED: The Nathan Drake Collection - PlayStation 4"


In [ ]:
# Provide natural language chat input and get item recommendations
INPUT = """I like scifi and action games.\n<|rec|>""".strip()
response = chat(INPUT)

09:11:35 - USER: I like scifi and action games.
<|rec|>
09:11:35 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

<|sid_start|><|sid_64|><|sid_313|><|sid_637|><|sid_768|><|sid_end|>, <|sid_start|><|sid_219|><|sid_274|><|sid_534|><|sid_768|><|sid_end|>, <|sid_start|><|sid_64|><|sid_447|><|sid_706|><|sid_768|><|sid_end|><|im_end|>
09:11:37 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
09:11:37 - ASSISTANT: "Halo 3 Limited Edition -Xbox 360", "Borderlands Ultimate Edition - PS3 [Digital Code]", "Lost Planet: Extreme Condition - Xbox 360"


In [ ]:
# Provide natural language chat input and get item recommendations
INPUT = """I like animal and cute games.\n<|rec|>""".strip()
response = chat(INPUT)

09:11:51 - USER: I like animal and cute games.
<|rec|>
09:11:51 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

<|sid_start|><|sid_173|><|sid_324|><|sid_764|><|sid_768|><|sid_end|>, <|sid_start|><|sid_201|><|sid_397|><|sid_738|><|sid_769|><|sid_end|>, <|sid_start|><|sid_173|><|sid_305|><|sid_670|><|sid_768|><|sid_end|><|im_end|>
09:11:53 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
09:11:53 - ASSISTANT: "Animal Crossing: New Leaf", "Disney Magical World - Nintendo 3DS", "Nintendogs + Cats: Golden Retriever and New Friends"


In [ ]:
# Provide item to get recommendation and explanation
INPUT = """I just finished <|sid_start|><|sid_125|><|sid_417|><|sid_656|><|sid_768|><|sid_end|>. Suggest another <|rec|> and explain why:""".strip()
response = chat(INPUT)

09:12:16 - USER: I just finished "Dragon Quest Heroes II Explorer's Edition - PlayStation 4". Suggest another <|rec|> and explain why:
09:12:16 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

<|sid_start|><|sid_139|><|sid_289|><|sid_534|><|sid_768|><|sid_end|>

If you liked Dragon Quest Heroes II, you might like Nights of Azure because both are action RPGs for the PlayStation 4 with a focus on combat and character progression. Both games offer a fantasy setting with a strong narrative, appealing to players who enjoy immersive storytelling and engaging gameplay mechanics within the same console ecosystem.<|im_end|>
09:12:22 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
09:12:22 - ASSISTANT: "Nights of Azure - PlayStation 4"

If you liked Dragon Quest Heroes II, you might like Nights of Azure because both are action RPGs for the PlayStation 4 with a focus on combat and character progression

In [ ]:
# Provide list of items and get suggested bundle title and description
INPUT = """Suggest a name and description for a bundle of <|sid_start|><|sid_8|><|sid_454|><|sid_630|><|sid_768|><|sid_end|>, <|sid_start|><|sid_219|><|sid_453|><|sid_620|><|sid_768|><|sid_end|>, <|sid_start|><|sid_219|><|sid_469|><|sid_585|><|sid_768|><|sid_end|>""".strip()
response = chat(INPUT)

09:12:45 - USER: Suggest a name and description for a bundle of "Resistance 2 Collector's Edition - Playstation 3", "PS3 Killzone Trilogy Collection - 2 Disc", "The Last of Us - PlayStation 3"
09:12:45 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

PS3 Action & Survival Shooter Essentials: Experience intense first-person and third-person shooter action with these critically acclaimed PS3 titles, offering immersive storytelling and thrilling combat.<|im_end|>
09:12:48 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
09:12:48 - ASSISTANT: PS3 Action & Survival Shooter Essentials: Experience intense first-person and third-person shooter action with these critically acclaimed PS3 titles, offering immersive storytelling and thrilling combat.


## Multi-Turn Conversations

In [ ]:
# 1st turn: Ask for games similar to Mario Kart
INPUT = "I'm looking for games similar to Mario Kart. <|rec|>"
response1 = chat(INPUT)

09:13:17 - USER: I'm looking for games similar to Mario Kart. <|rec|>
09:13:17 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

<|sid_start|><|sid_131|><|sid_503|><|sid_738|><|sid_768|><|sid_end|>, <|sid_start|><|sid_131|><|sid_260|><|sid_577|><|sid_768|><|sid_end|>, <|sid_start|><|sid_145|><|sid_416|><|sid_671|><|sid_768|><|sid_end|><|im_end|>
09:13:19 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
09:13:19 - ASSISTANT: "Need for Speed Most Wanted - Sony PSP", "Need for Speed Hot Pursuit 2 - PlayStation 2", "Burnout Legends - Nintendo DS"


In [ ]:
# 2nd turn: Tweak it for Xbox
INPUT = "How about something similar but for Xbox? <|rec|>"
response2 = chat(INPUT, new_convo=False)

09:13:34 - ========================================= Conversation History =========================================
09:13:34 - [Turn 1] USER: I'm looking for games similar to Mario Kart. <|rec|>
09:13:34 - [Turn 1] ASSISTANT: "Need for Speed Most Wanted - Sony PSP", "Need for Speed Hot Pursuit 2 - PlayStation 2", "Burnout Legends - Nintendo DS"
09:13:34 - =================================== Current Turn ===================================
09:13:34 - USER: How about something similar but for Xbox? <|rec|>
09:13:34 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

<|sid_start|><|sid_183|><|sid_461|><|sid_517|><|sid_768|><|sid_end|>, <|sid_start|><|sid_183|><|sid_461|><|sid_562|><|sid_768|><|sid_end|>, <|sid_start|><|sid_183|><|sid_461|><|sid_616|><|sid_768|><|sid_end|><|im_end|>
09:13:36 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
09:13:36 - ASSISTANT: "Need for Speed Carbon - Xbox 360", "P

In [ ]:
# 3rd turn: Ask for bundle name
INPUT = "Suggest a name and description for the bundle"
response3 = chat(INPUT, new_convo=False)

09:13:51 - ========================================= Conversation History =========================================
09:13:51 - [Turn 1] USER: I'm looking for games similar to Mario Kart. <|rec|>
09:13:51 - [Turn 1] ASSISTANT: "Need for Speed Most Wanted - Sony PSP", "Need for Speed Hot Pursuit 2 - PlayStation 2", "Burnout Legends - Nintendo DS"
09:13:51 - [Turn 2] USER: How about something similar but for Xbox? <|rec|>
09:13:51 - [Turn 2] ASSISTANT: "Need for Speed Carbon - Xbox 360", "Project Gotham Racing 3 - Xbox 360", "Forza Motorsport 4 - Xbox 360"
09:13:51 - =================================== Current Turn ===================================
09:13:51 - USER: Suggest a name and description for the bundle
09:13:51 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

Xbox Racing Legends: Speed & Style Pack<|im_end|>
09:13:52 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
09:13:52 - ASSISTANT

## More multi-turn conversation

In [ ]:
# 1st turn: Xbox games similar to Zelda
INPUT = """Recommend Xbox games similar to <|sid_start|><|sid_201|><|sid_396|><|sid_608|><|sid_769|><|sid_end|>\n<|rec|>""".strip()
response = chat(INPUT)

09:14:07 - USER: Recommend Xbox games similar to "The Legend of Zelda: Ocarina of Time 3D"
<|rec|>
09:14:07 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

<|sid_start|><|sid_64|><|sid_271|><|sid_576|><|sid_768|><|sid_end|>, <|sid_start|><|sid_167|><|sid_279|><|sid_746|><|sid_768|><|sid_end|>, <|sid_start|><|sid_167|><|sid_453|><|sid_548|><|sid_768|><|sid_end|><|im_end|>
09:14:09 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
09:14:09 - ASSISTANT: "Fallout: New Vegas - Xbox 360 Ultimate Edition", "Lightning Returns: Final Fantasy XIII - Xbox 360", "Doom 3 - Xbox 360 BFG Edition"


In [ ]:
# 2nd turn: Making it multiplayer
INPUT = "What about something similar but for multiplayer on Wii?\n<|rec|>"
response = chat(INPUT, new_convo=False)

09:14:20 - ========================================= Conversation History =========================================
09:14:20 - [Turn 1] USER: Recommend Xbox games similar to "The Legend of Zelda: Ocarina of Time 3D"
<|rec|>
09:14:20 - [Turn 1] ASSISTANT: "Fallout: New Vegas - Xbox 360 Ultimate Edition", "Lightning Returns: Final Fantasy XIII - Xbox 360", "Doom 3 - Xbox 360 BFG Edition"
09:14:20 - =================================== Current Turn ===================================
09:14:20 - USER: What about something similar but for multiplayer on Wii?
<|rec|>
09:14:20 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

<|sid_start|><|sid_91|><|sid_431|><|sid_741|><|sid_768|><|sid_end|>, <|sid_start|><|sid_91|><|sid_435|><|sid_669|><|sid_768|><|sid_end|>, <|sid_start|><|sid_91|><|sid_435|><|sid_669|><|sid_768|><|sid_end|><|im_end|>
09:14:22 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
09:14:

In [ ]:
# 2nd turn: Making it multiplayer
INPUT = "Suggest similar games but for racing\n<|rec|>"
response = chat(INPUT, new_convo=False)

09:14:43 - ========================================= Conversation History =========================================
09:14:43 - [Turn 1] USER: Recommend Xbox games similar to "The Legend of Zelda: Ocarina of Time 3D"
<|rec|>
09:14:43 - [Turn 1] ASSISTANT: "Fallout: New Vegas - Xbox 360 Ultimate Edition", "Lightning Returns: Final Fantasy XIII - Xbox 360", "Doom 3 - Xbox 360 BFG Edition"
09:14:43 - [Turn 2] USER: What about something similar but for multiplayer on Wii?
<|rec|>
09:14:43 - [Turn 2] ASSISTANT: "Pictionary - Udraw - Nintendo Wii", "Angry Birds Trilogy - Nintendo Wii", "Angry Birds Trilogy - Nintendo Wii"
09:14:43 - =================================== Current Turn ===================================
09:14:43 - USER: Suggest similar games but for racing
<|rec|>
09:14:43 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

<|sid_start|><|sid_183|><|sid_461|><|sid_616|><|sid_768|><|sid_end|>, <|sid_start|><|sid_183|><|sid_461|